In [1]:
# Importing Required Libraries
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px


In [5]:
# Loading data into dataframe
data = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
)
data.head()

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.50,27483.571,1558,7,60.223,0.010000,5.4,456.0,Supperminicar,Georgia
1,2/29/1980,1980,Feb,1,98.75,0.75,24308.678,3048,4,45.986,-0.309594,4.8,555.9,Supperminicar,New York
2,3/31/1980,1980,Mar,1,107.48,0.20,28238.443,3137,3,35.141,-0.308614,3.4,620.0,Mediumfamilycar,New York
3,4/30/1980,1980,Apr,1,115.01,1.00,32615.149,1653,7,45.673,0.230596,4.2,702.8,Supperminicar,Illinois
4,5/31/1980,1980,May,1,98.72,0.20,23829.233,1319,4,52.997,0.138197,5.3,770.4,Smallfamiliycar,California


In [6]:
# Initializing app
app = dash.Dash(__name__, suppress_callback_exceptions=True)
app.title = "Automobile Statistics Dashboard"


In [10]:
# Total years in the dataset
year_list = sorted(data["Year"].unique())


In [11]:
# Layout of the app
app.layout = html.Div(
    [
        html.H1(
            "Automobile Sales Statistics Dashboard",
            style={"textAlign": "center", "color": "#2c3e50", "marginBottom": "30px"},
        ),
        dcc.Tabs(
            id="tabs",
            value="Yearly Statistics",
            children=[
                dcc.Tab(label="Yearly Statistics", value="Yearly Statistics"),
                dcc.Tab(
                    label="Recession Period Statistics",
                    value="Recession Period Statistics",
                ),
            ],
            style={"fontSize": "18px"},
        ),
        html.Div(id="controls-container"),
        html.Div(id="charts-container"),
    ],
    style={"padding": "20px", "fontFamily": "Arial, sans-serif"},
)

In [12]:
# Callback for controls based on tab selection
@app.callback(Output("controls-container", "children"), Input("tabs", "value"))
def render_controls(tab):
    if tab == "Yearly Statistics":
        return html.Div(
            [
                html.Label("Select a Year:"),
                dcc.Dropdown(
                    id="year-dropdown",
                    options=[{"label": str(year), "value": year} for year in year_list],
                    value=year_list[0],
                    style={"width": "50%"},
                ),
            ]
        )
    return None

In [13]:
# Callback to render charts
@app.callback(
    Output("charts-container", "children"),
    [Input("tabs", "value"), Input("year-dropdown", "value")],
    prevent_initial_call=True,
)
def render_charts(tab, selected_year):
    if tab == "Yearly Statistics" and selected_year:
        yearly_data = data[data["Year"] == selected_year]

        yas = data.groupby("Year")["Automobile_Sales"].mean().reset_index()
        monthly_sales = (
            yearly_data.groupby("Month")["Automobile_Sales"].sum().reset_index()
        )
        avr_vdata = (
            yearly_data.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index()
        )
        exp_data = (
            yearly_data.groupby("Vehicle_Type")["Advertising_Expenditure"]
            .sum()
            .reset_index()
        )

        return html.Div(
            [
                html.Div(
                    [
                        dcc.Graph(
                            figure=px.line(
                                yas,
                                x="Year",
                                y="Automobile_Sales",
                                title="Yearly Average Sales",
                            )
                        )
                    ],
                    className="card",
                ),
                html.Div(
                    [
                        dcc.Graph(
                            figure=px.line(
                                monthly_sales,
                                x="Month",
                                y="Automobile_Sales",
                                title=f"Monthly Sales in {selected_year}",
                            )
                        )
                    ],
                    className="card",
                ),
                html.Div(
                    [
                        dcc.Graph(
                            figure=px.bar(
                                avr_vdata,
                                x="Vehicle_Type",
                                y="Automobile_Sales",
                                title="Avg. Vehicles Sold by Type",
                            )
                        )
                    ],
                    className="card",
                ),
                html.Div(
                    [
                        dcc.Graph(
                            figure=px.pie(
                                exp_data,
                                values="Advertising_Expenditure",
                                names="Vehicle_Type",
                                title="Ad Expenditure by Vehicle Type",
                            )
                        )
                    ],
                    className="card",
                ),
            ],
            style={"display": "grid", "gridTemplateColumns": "1fr 1fr", "gap": "20px"},
        )

    elif tab == "Recession Period Statistics":
        recession_data = data[data["Recession"] == 1]

        yearly_rec = (
            recession_data.groupby("Year")["Automobile_Sales"].mean().reset_index()
        )
        average_sales = (
            recession_data.groupby("Vehicle_Type")["Automobile_Sales"]
            .mean()
            .reset_index()
        )
        exp_rec = (
            recession_data.groupby("Vehicle_Type")["Advertising_Expenditure"]
            .sum()
            .reset_index()
        )
        unemployment_data = (
            recession_data.groupby("Vehicle_Type")["unemployment_rate"]
            .mean()
            .reset_index()
        )

        return html.Div(
            [
                html.Div(
                    [
                        dcc.Graph(
                            figure=px.line(
                                yearly_rec,
                                x="Year",
                                y="Automobile_Sales",
                                title="Sales During Recession Years",
                            )
                        )
                    ],
                    className="card",
                ),
                html.Div(
                    [
                        dcc.Graph(
                            figure=px.bar(
                                average_sales,
                                x="Vehicle_Type",
                                y="Automobile_Sales",
                                title="Avg Sales by Type During Recession",
                            )
                        )
                    ],
                    className="card",
                ),
                html.Div(
                    [
                        dcc.Graph(
                            figure=px.pie(
                                exp_rec,
                                values="Advertising_Expenditure",
                                names="Vehicle_Type",
                                title="Ad Spend During Recession",
                            )
                        )
                    ],
                    className="card",
                ),
                html.Div(
                    [
                        dcc.Graph(
                            figure=px.bar(
                                unemployment_data,
                                x="Vehicle_Type",
                                y="unemployment_rate",
                                title="Unemployment Rate by Vehicle Type",
                            )
                        )
                    ],
                    className="card",
                ),
            ],
            style={"display": "grid", "gridTemplateColumns": "1fr 1fr", "gap": "20px"},
        )

    return None

In [14]:
# CSS-style card-like layout (if using external CSS this can be removed)
app.index_string = """
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            .card {
                background-color: #f9f9f9;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 2px 6px rgba(0,0,0,0.1);
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
"""

In [15]:
# Run app (Dashboard)
if __name__ == "__main__":
    app.run(debug=True)